In [ ]:
import eco2ai
tracker = eco2ai.Tracker(project_name="lct_12_sber_multimodal_recsys", 
                         experiment_description="geo_pyspark_v1_1_train_test", ignore_warnings=True)
tracker.start()

In [1]:
import os
os.environ["OMP_NUM_THREADS"] = "4"#4

import pandas as pd
import numpy as np
import torch
from functools import partial
import pytorch_lightning as pl
import warnings
import pickle
import time
warnings.filterwarnings("ignore")

from torch.utils.data import DataLoader

from ptls.data_load.datasets import MemoryMapDataset
from ptls.data_load.iterable_processing.iterable_seq_len_limit import ISeqLenLimit
from ptls.data_load.iterable_processing.to_torch_tensor import ToTorch
from ptls.data_load.iterable_processing.feature_filter import FeatureFilter
from ptls.nn import TrxEncoder, RnnSeqEncoder
from ptls.frames.coles import CoLESModule
from ptls.data_load.iterable_processing import SeqLenFilter
from ptls.frames.coles import ColesIterableDataset, ColesDataset
from ptls.frames.coles.split_strategy import SampleSlices
from ptls.frames import PtlsDataModule
from ptls.preprocessing import PandasDataPreprocessor, PysparkDataPreprocessor
from ptls.data_load.utils import collate_feature_dict
from ptls.data_load.iterable_processing_dataset import IterableProcessingDataset
from ptls.data_load.datasets import ParquetDataset, ParquetFiles
import ptls

from tqdm.auto import tqdm
#import lightgbm as ltb
from datetime import datetime

import multiprocessing
print('cpu', multiprocessing.cpu_count())

import psutil    
print('ram', round(psutil.virtual_memory().total/1024/1024/1024,1), 'gb')  # total physical memory in Bytes

debug=False

print('torch', torch.__version__)
print('pandas', pd.__version__)
print('numpy', np.__version__)
print('pytorch-lightning', pl.__version__)
print(time.ctime())

cpu 8
ram 94.3 gb
torch 1.13.1+cu116
Mon Jun 10 05:59:41 2024


In [2]:
!nvidia-smi

Mon Jun 10 05:59:41 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-PCIE...  On   | 00000000:8B:00.0 Off |                    0 |
| N/A   30C    P0    24W / 250W |      4MiB / 32768MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from functools import reduce
from pyspark.sql import DataFrame
import pyspark
import os
print('pyspark', pyspark.__version__)
os.environ["PYARROW_IGNORE_TIMEZONE"] = "1" 
# без этой строчки у нас будет возникать постоянное предупреждение с просьбой установить эту переменную в значение 1, что мы заранее и делаем

In [4]:
spark_conf = pyspark.SparkConf()
spark_conf.setMaster("local[*]").setAppName("PysparkDataPreprocessor")
spark_conf.set("spark.driver.maxResultSize", "4g")
spark_conf.set("spark.executor.memory", "16g")
spark_conf.set("spark.executor.memoryOverhead", "4g")
spark_conf.set("spark.driver.memory", "16g")
spark_conf.set("spark.driver.memoryOverhead", "4g")
spark_conf.set("spark.cores.max", "8")
spark_conf.set("spark.sql.shuffle.partitions", "200")
spark_conf.set("spark.local.dir", "../../spark_local_dir")
spark_conf.set("spark.executor.heartbeatInterval", "200000")
spark_conf.set("spark.network.timeout", "300000")


spark = SparkSession.builder.config(conf=spark_conf).getOrCreate()
spark.sparkContext.getConf().getAll()

24/06/10 05:59:45 WARN Utils: Your hostname, gpupreemnew resolves to a loopback address: 127.0.1.1; using 10.129.0.19 instead (on interface eth0)
24/06/10 05:59:45 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/10 05:59:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/06/10 05:59:46 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).


[('spark.executor.extraJavaOptions',
  '-Djava.net.preferIPv6Addresses=false -XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED --add-opens=java.security.jgss/sun.security.krb5=ALL-UNNAMED -Djdk.reflect.useDirectMethodHandle=false'),
 ('spark.driver.memoryOverhead', '4g'),
 ('spark.app.submitTime', '1717999185784'),
 ('spark.app.id', 'local-1717999187662'),
 ('spark.local.dir', '../.

## 1. Предобратока данных

In [5]:
%%time
geo_train = spark.read.parquet("Hackathon/geo_train.parquet")
geo_test = spark.read.parquet("Hackathon/geo_test.parquet")

CPU times: user 7.43 ms, sys: 544 µs, total: 7.98 ms
Wall time: 2.91 s


In [6]:
print(geo_train.count(), geo_test.count())

CPU times: user 2.67 ms, sys: 68 µs, total: 2.74 ms
Wall time: 144 ms


In [7]:
%%time
if debug:
    train_clients = [i['client_id'] for i in geo_train.select('client_id').distinct().collect()]
    print(len(train_clients))
    train_clients = pd.Series(train_clients).sample(random_state=37, frac=0.1).values.tolist()
    print(len(train_clients))
    geo_train = geo_train.filter(col('client_id').isin(train_clients))
    #print(transactions_train['client_id'].n_unique())

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.96 µs


In [8]:
%%time
if debug:
    test_clients = [i['client_id'] for i in geo_test.select('client_id').distinct().collect()]
    print(len(test_clients))
    test_clients = pd.Series(test_clients).sample(random_state=37, frac=0.1).values.tolist()
    print(len(test_clients))
    geo_test = geo_test.filter(col('client_id').isin(test_clients))
    #print(transactions_test['client_id'].n_unique())

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 6.91 µs


In [9]:
print(geo_train.count(), geo_test.count())
#transactions_train.show(5)

528278383 139047888


In [10]:
# объединяем train и test, чтобы историческая информация о поведении тестовых пользователей тоже использовалась для обучения
geo_train = reduce(DataFrame.unionAll, [geo_train, geo_test])
print(geo_train.count())

667326271


In [11]:
%%time

geo_preprocessor = PysparkDataPreprocessor(
    col_id="client_id",
    col_event_time="event_time",
    event_time_transformation="dt_to_timestamp",
    cols_category=["geohash_4",
                   "geohash_5",
                   "geohash_6"],
    #cols_identity="amount",
    #return_records=False,
)

processed_geo_train = geo_preprocessor.fit_transform(geo_train)
processed_geo_test = geo_preprocessor.transform(geo_test)

24/06/09 15:49:31 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/09 15:49:31 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/09 15:49:31 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/09 15:49:47 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/09 15:49:47 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/09 15:49:48 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/09 1

CPU times: user 23.2 s, sys: 9.5 s, total: 32.7 s
Wall time: 2min 43s


In [12]:
processed_geo_train.dtypes

[('client_id', 'string'),
 ('event_time', 'array<bigint>'),
 ('geohash_4', 'array<int>'),
 ('geohash_5', 'array<int>'),
 ('geohash_6', 'array<int>')]

In [13]:
with open('geo_preprocessor_pyspark_v1.pickle', 'wb') as f:
    pickle.dump(geo_preprocessor, f)

In [14]:
%%time
processed_geo_train.write.parquet('processed_geo_train_pyspark_v1.parquet', mode='overwrite')
processed_geo_test.write.parquet('processed_geo_test_pyspark_v1.parquet', mode='overwrite')

24/06/09 17:14:29 WARN DAGScheduler: Broadcasting large task binary with size 1095.6 KiB
24/06/09 17:33:23 WARN DAGScheduler: Broadcasting large task binary with size 1088.6 KiB


CPU times: user 1.36 s, sys: 624 ms, total: 1.98 s
Wall time: 1h 41min 27s


In [15]:
print(processed_geo_train.count(), processed_geo_test.count())
#processed_train.show(5)

789051 166747


24/06/09 17:40:15 WARN JavaUtils: Attempt to delete using native Unix OS command failed for path = /home/jovyan/spark_local_dir/blockmgr-d3100dd9-9f8a-4e71-bc53-a7d3ffd7ddf2. Falling back to Java IO way
java.io.IOException: Failed to delete: /home/jovyan/spark_local_dir/blockmgr-d3100dd9-9f8a-4e71-bc53-a7d3ffd7ddf2
	at org.apache.spark.network.util.JavaUtils.deleteRecursivelyUsingUnixNative(JavaUtils.java:173)
	at org.apache.spark.network.util.JavaUtils.deleteRecursively(JavaUtils.java:109)
	at org.apache.spark.network.util.JavaUtils.deleteRecursively(JavaUtils.java:90)
	at org.apache.spark.util.SparkFileUtils.deleteRecursively(SparkFileUtils.scala:121)
	at org.apache.spark.util.SparkFileUtils.deleteRecursively$(SparkFileUtils.scala:120)
	at org.apache.spark.util.Utils$.deleteRecursively(Utils.scala:1126)
	at org.apache.spark.storage.DiskBlockManager.$anonfun$doStop$1(DiskBlockManager.scala:368)
	at org.apache.spark.storage.DiskBlockManager.$anonfun$doStop$1$adapted(DiskBlockManager.sc

In [ ]:
del processed_geo_train, processed_geo_test

## 2. Тренировка модели

In [5]:
with open('geo_preprocessor_pyspark_v1.pickle', 'rb') as f:
    geo_preprocessor = pickle.load(f)

**Обработка датасета:**

- Транзакции, у которых размер < min_seq_len выкидываются
- Транзакции, у которых длина > max_seq_len, обрезаются и конвертируются в torch.tensor
- Не нужные для CoLES фичи удаляются

In [9]:
%%time
train = MemoryMapDataset(
    data=ParquetDataset(ParquetFiles('processed_geo_train_pyspark_v1.parquet')),#processed_train.toPandas().to_dict("records"),
    i_filters=[
        FeatureFilter(drop_feature_names=['client_id', 'target_1', 'target_2', 'target_3', 'target_4']),
        SeqLenFilter(min_seq_len=32),
        ISeqLenLimit(max_seq_len=4096),
        ToTorch()
    ]
)

CPU times: user 1min 41s, sys: 29 s, total: 2min 10s
Wall time: 1min 38s


In [10]:
%%time
test = MemoryMapDataset(
    data=ParquetDataset(ParquetFiles('processed_geo_test_pyspark_v1.parquet')),#processed_test.toPandas().to_dict("records"),
    i_filters=[
        FeatureFilter(drop_feature_names=['client_id', 'target_1', 'target_2', 'target_3', 'target_4']),
        SeqLenFilter(min_seq_len=32),
        ISeqLenLimit(max_seq_len=4096),
        ToTorch()
    ]
)

CPU times: user 22.6 s, sys: 7.23 s, total: 29.9 s
Wall time: 21.5 s


In [11]:
%%time
train_ds = ColesDataset(
    data=train,
    splitter=SampleSlices(
        split_count=5,
        cnt_min=32,
        cnt_max=180
    )
)

valid_ds = ColesDataset(
    data=test,
    splitter=SampleSlices(
        split_count=5,
        cnt_min=32,
        cnt_max=180
    )
)

CPU times: user 29 µs, sys: 8 µs, total: 37 µs
Wall time: 41.7 µs


In [12]:
%%time
train_dl = PtlsDataModule(
    train_data=train_ds,
    train_num_workers=16,
    train_batch_size=256,
    valid_data=valid_ds,
    valid_num_workers=16,
    valid_batch_size=256
)

CPU times: user 332 µs, sys: 97 µs, total: 429 µs
Wall time: 436 µs


- количественных фич (numeric_values) в этом датасете нет, так что используем только категориальные (embedidngs - nn.Embedidngs)

In [6]:
%%time
trx_encoder_params = dict(
    embeddings_noise=0.003,
    #numeric_values={'amount': 'log'},
    embeddings={
        "geohash_4": {'in': geo_preprocessor.get_category_dictionary_sizes()["geohash_4"], "out": 32},
        "geohash_5": {'in': geo_preprocessor.get_category_dictionary_sizes()["geohash_5"], "out": 32},
        'geohash_6': {'in': geo_preprocessor.get_category_dictionary_sizes()["geohash_6"], 'out': 32},
      }
)

CPU times: user 18 µs, sys: 5 µs, total: 23 µs
Wall time: 26.2 µs


- **TrxEncoder** - обрабатывает каждую геолокацию (строит для неё эмбеддиг)
- **SeqEncoder** - обрабатывает последовательность

In [7]:
%%time
seq_encoder = RnnSeqEncoder(
    trx_encoder=TrxEncoder(**trx_encoder_params),
    hidden_size=256,
    type='gru',
)

CPU times: user 32.6 ms, sys: 19.3 ms, total: 51.9 ms
Wall time: 109 ms


In [8]:
%%time
model = CoLESModule(
    seq_encoder=seq_encoder,
    optimizer_partial=partial(torch.optim.Adam, lr=0.001),
    lr_scheduler_partial=partial(torch.optim.lr_scheduler.StepLR, step_size=3, gamma=0.9025)
)

CPU times: user 30.2 ms, sys: 11 ms, total: 41.2 ms
Wall time: 96.6 ms


In [13]:
%%time
trainer = pl.Trainer(
    max_epochs=25,
    limit_val_batches=5000,
    devices=[0],
    enable_progress_bar=True,
    gradient_clip_val=0.5,
    logger=pl.loggers.TensorBoardLogger(
        save_dir='./logdir',
        name='geo_pyspark_v1_1_result'
    ),
    callbacks=[
        pl.callbacks.LearningRateMonitor(logging_interval='step'),
        pl.callbacks.ModelCheckpoint(every_n_train_steps=5000, save_top_k=-1),
    ]
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


CPU times: user 311 ms, sys: 53.8 ms, total: 364 ms
Wall time: 349 ms


In [14]:
%%time
trainer.fit(model, train_dl)

Missing logger folder: ./logdir/geo_pyspark_v1_1_result
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name               | Type            | Params
-------------------------------------------------------
0 | _loss              | ContrastiveLoss | 0     
1 | _seq_encoder       | RnnSeqEncoder   | 1.2 M 
2 | _validation_metric | BatchRecallTopK | 0     
3 | _head              | Head            | 0     
-------------------------------------------------------
1.2 M     Trainable params
0         Non-trainable params
1.2 M     Total params
4.929     Total estimated model params size (MB)


Sanity Checking: |                                        | 0/? [00:00<?, ?it/s]

Training: |                                               | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

...

Validation: |                                             | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=25` reached.


CPU times: user 2h 21min 47s, sys: 8min 45s, total: 2h 30min 33s
Wall time: 2h 32min 10s


In [15]:
torch.save(model.state_dict(), './model_geo_pyspark_v1_1.pt')

## 3. Inference

Для каждого пользователя известно 12 таргетов, инференс происходит следующим образом:

Чтобы не происходило лика нужно для каждого клиента делать срез до текущего месяца:

Берутся все геолокации за первый месяц, им соответствует 1-ый таргет из 12,
потом берутся геолокации за первый и второй месяц пользователя и им соотвествует 2-ой таргет и так далее.
То есть для данного пользователя, имеющего геолокации за год, мы можем получить 12 эмбеддингов, каждому из которых соответствует 1 таргет

In [16]:
class GetSplit(IterableProcessingDataset):
    def __init__(
        self,
        start_month,
        end_month,
        year=2022,
        col_id='client_id',
        col_time='event_time'
    ):
        super().__init__()
        self.start_month = start_month
        self.end_month = end_month
        self._year = year
        self._col_id = col_id
        self._col_time = col_time

    def __iter__(self):
        for rec in self._src:
            for month in range(self.start_month, self.end_month+1):
                features = rec[0] if type(rec) is tuple else rec
                features = features.copy()

                if month == 12:
                    month_event_time = datetime(self._year + 1, 1, 1).timestamp()
                else:
                    month_event_time = datetime(self._year, month + 1, 1).timestamp()

                year_event_time = datetime(self._year, 1, 1).timestamp()

                mask = features[self._col_time] < month_event_time

                for key, tensor in features.items():
                    if key.startswith('target'):
                        features[key] = tensor[month - 1].tolist()
                    elif key != self._col_id:
                        features[key] = tensor[mask]

                features[self._col_id] += '_month=' + str(month)

                yield features

def collate_feature_dict_with_target(batch, col_id='client_id', targets=False):
    batch_ids = []
    target_cols = []
    for sample in batch:
        batch_ids.append(sample[col_id])
        del sample[col_id]

        if targets:
            target_cols.append([sample[f'target_{i}'] for i in range(1, 5)])
            del sample['target_1']
            del sample['target_2']
            del sample['target_3']
            del sample['target_4']

    padded_batch = collate_feature_dict(batch)
    if targets:
        return padded_batch, batch_ids, target_cols
    return padded_batch, batch_ids


class InferenceModuleMultimodal(pl.LightningModule):
    def __init__(self, model, pandas_output=True, drop_seq_features=True, model_out_name='out'):
        super().__init__()

        self.model = model
        self.pandas_output = pandas_output
        self.drop_seq_features = drop_seq_features
        self.model_out_name = model_out_name

    def forward(self, x):
        x_len = len(x)
        if x_len == 3:
            x, batch_ids, target_cols = x
        else:
            x, batch_ids = x

        out = self.model(x)
        if x_len == 3:
            target_cols = torch.tensor(target_cols)
            x_out = {
                'client_id': batch_ids,
                'target_1': target_cols[:, 0],
                'target_2': target_cols[:, 1],
                'target_3': target_cols[:, 2],
                'target_4': target_cols[:, 3],
                self.model_out_name: out
            }
        else:
            x_out = {
                'client_id': batch_ids,
                self.model_out_name: out
            }
        torch.cuda.empty_cache()

        if self.pandas_output:
            return self.to_pandas(x_out)
        return x_out

    @staticmethod
    def to_pandas(x):
        expand_cols = []
        scalar_features = {}

        for k, v in x.items():
            if type(v) is torch.Tensor:
                v = v.cpu().numpy()

            if type(v) is list or len(v.shape) == 1:
                scalar_features[k] = v
            elif len(v.shape) == 2:
                expand_cols.append(k)
            else:
                scalar_features[k] = None

        dataframes = [pd.DataFrame(scalar_features)]
        for col in expand_cols:
            v = x[col].cpu().numpy()
            dataframes.append(pd.DataFrame(v, columns=[f'{col}_{i:04d}' for i in range(v.shape[1])]))

        return pd.concat(dataframes, axis=1)

In [17]:
model.load_state_dict(torch.load('model_geo_pyspark_v1_1.pt'))

In [18]:
%%time
test = MemoryMapDataset(
    data=ParquetDataset(ParquetFiles('processed_geo_test_pyspark_v1.parquet')),#processed_test.toPandas().to_dict("records"),
    i_filters=[
        ISeqLenLimit(max_seq_len=4096),
        FeatureFilter(keep_feature_names=['client_id', 'target_1', 'target_2', 'target_3', 'target_4']),
        ToTorch(),
    ]
)

CPU times: user 23.3 s, sys: 7.33 s, total: 30.7 s
Wall time: 20.8 s


In [19]:
inference_test_dl = DataLoader(
        dataset=test,
        collate_fn=collate_feature_dict_with_target,
        shuffle=False,
        num_workers=0,
        batch_size=256,
    )

In [20]:
inf_module = InferenceModuleMultimodal(
        model=model,
        pandas_output=True,
        drop_seq_features=True,
        model_out_name='emb',
    )

In [21]:
trainer = pl.Trainer(devices=[0], max_epochs=-1)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [22]:
inf_test_embeddings = pd.concat(
        trainer.predict(inf_module, inference_test_dl)
    )
inf_test_embeddings.to_parquet("test_geo_pyspark_v1_1.parquet", index=False, engine="pyarrow", compression="snappy")

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |                                             | 0/? [00:00<?, ?it/s]

In [23]:
print(inf_test_embeddings.shape)
inf_test_embeddings.head(2)

(166747, 257)


,client_id,emb_0000,emb_0001,emb_0002,emb_0003,emb_0004,emb_0005,emb_0006,emb_0007,emb_0008,...,emb_0246,emb_0247,emb_0248,emb_0249,emb_0250,emb_0251,emb_0252,emb_0253,emb_0254,emb_0255
0,001ea97e38c971ed474a17325f3ffad49187b1ffc3a624...,-0.038996,0.506409,-0.009662,-0.791979,0.017973,0.768293,0.901419,-0.664030,0.693993,...,-0.036012,0.638229,0.283614,0.133935,-0.126021,0.213060,0.831937,-0.857459,-0.823549,-0.731591
1,00296f9023283991f09082088a7c0ca3ea0752ee486988...,0.105085,0.661438,-0.051763,-0.727864,0.089072,0.821102,0.817634,-0.398612,0.781941,...,-0.788290,0.258295,0.783549,0.380403,-0.105244,0.270898,0.795662,-0.816093,-0.491044,-0.871947


Файл **sample_submission** составляется из **client_id** файла **test_target_b**. Так как не у всех пользователей может быть транзакционная история, мы для простоты заполняем их фичи нулями.

In [24]:
%%time
test_target_b = pd.read_parquet("Hackathon/test_target_b.parquet")

CPU times: user 481 ms, sys: 242 ms, total: 723 ms
Wall time: 937 ms


In [25]:
not_only_trx = pd.DataFrame({"client_id": test_target_b["client_id"].unique()}).merge(inf_test_embeddings, how="left").fillna(0)
not_only_trx

,client_id,emb_0000,emb_0001,emb_0002,emb_0003,emb_0004,emb_0005,emb_0006,emb_0007,emb_0008,...,emb_0246,emb_0247,emb_0248,emb_0249,emb_0250,emb_0251,emb_0252,emb_0253,emb_0254,emb_0255
0,2b7ff0c1c99cefe259ed83c5dfa0a403f2cbc88032b671...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0433d23e224b7a520656da6181efadb8d556bb293158c9...,-0.006920,0.371985,0.105188,-0.940242,0.234232,0.853081,0.907563,0.432888,0.698880,...,-0.844448,0.070265,0.970539,0.481419,0.136677,0.149464,0.859307,-0.867117,-0.871099,-0.803203
2,f2ce8b292e5f9f778f3e20db7608ac76dc8812113a2631...,-0.035280,0.719363,-0.276090,-0.913701,0.028656,0.934402,0.987570,-0.770712,0.580129,...,-0.894032,-0.348200,0.325509,-0.341641,-0.250081,0.198994,0.649047,-0.868058,-0.913276,-0.867098
3,4f807e8b163c653bcaeff9f925983568f4c3e6b1a1f231...,0.128453,0.791124,0.297852,-0.891602,0.162554,0.843251,0.906598,-0.822932,0.916050,...,-0.847274,0.618208,0.947345,0.383501,-0.263884,-0.063679,0.703320,-0.909131,-0.300937,-0.747693
4,64369f6f8ae1b719332ee1bfb2b454e642b2053d2c9b8a...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140483,d49a66825bb16ceb5b6a01126e1f2391b085dba8da44ee...,0.024971,0.688765,0.427061,-0.577256,0.010310,0.633779,0.729747,-0.387141,0.782867,...,-0.711531,0.793534,0.943521,0.106403,-0.316784,0.169888,0.849713,-0.742480,-0.506382,-0.664590
140484,f772af6720c0b591d49b97946c5e420c1c077affc0f7c7...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
140485,06b282335bc4853f888e1ab50a6ba23a8e420d42313959...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
140486,90d423a25d7cdaf674f7d78bc37d88830443ff17717e02...,0.231699,-0.103652,-0.439368,-0.422195,0.181065,0.910682,0.835959,-0.441267,0.403910,...,-0.801953,0.477283,0.424229,0.064094,-0.052436,-0.138639,0.700406,-0.203674,-0.635084,-0.930173


In [26]:
not_only_trx.to_parquet("test_geo_pyspark_v1_1.parquet", index=False, engine="pyarrow", compression="snappy")

In [27]:
print(not_only_trx.shape)
not_only_trx.head(2)

(140488, 257)


,client_id,emb_0000,emb_0001,emb_0002,emb_0003,emb_0004,emb_0005,emb_0006,emb_0007,emb_0008,...,emb_0246,emb_0247,emb_0248,emb_0249,emb_0250,emb_0251,emb_0252,emb_0253,emb_0254,emb_0255
0,2b7ff0c1c99cefe259ed83c5dfa0a403f2cbc88032b671...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0433d23e224b7a520656da6181efadb8d556bb293158c9...,-0.00692,0.371985,0.105188,-0.940242,0.234232,0.853081,0.907563,0.432888,0.69888,...,-0.844448,0.070265,0.970539,0.481419,0.136677,0.149464,0.859307,-0.867117,-0.871099,-0.803203


In [28]:
not_only_trx.client_id.values[0]

'2b7ff0c1c99cefe259ed83c5dfa0a403f2cbc88032b671d0bcb425e90932db07'

In [29]:
del inf_test_embeddings

In [30]:
for j in [1, 3, 5, 7, 9, 11]:
    print('='*80)
    print(j)
    print(time.ctime())

    train = MemoryMapDataset(
        #data=processed_train.merge(processed_target9.drop(["event_time", 'target_len','new_mon'], axis=1), 
        #                           on="client_id", how="inner").to_dict("records"),
        ParquetDataset(ParquetFiles('processed_geo_train_pyspark_v1.parquet')),
        
        i_filters=[
            ISeqLenLimit(max_seq_len=4096),
            FeatureFilter(keep_feature_names=['client_id', 'target_1', 'target_2', 'target_3', 'target_4']),#
            GetSplit(start_month=j, end_month=j+1),
            ToTorch(),
        ]
    )

    inference_train_dl = DataLoader(
        dataset=train,
        #collate_fn=partial(collate_feature_dict_with_target, targets=True),
    collate_fn=collate_feature_dict_with_target,
        shuffle=False,
        num_workers=0,
        batch_size=256,
    )

    inf_train_embeddings = pd.concat(trainer.predict(inf_module, inference_train_dl))

    inf_train_embeddings.to_parquet("train_geo_pyspark_v1_1_m_"+str(j)+'_'+str(j+1)+".parquet", index=False, engine="pyarrow", compression="snappy")
    print(inf_train_embeddings.shape)
    del inf_train_embeddings, train

1
Mon Jun 10 08:36:51 2024


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |                                             | 0/? [00:00<?, ?it/s]

(1578102, 257)
3
Mon Jun 10 08:47:04 2024


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


...

Predicting: |                                             | 0/? [00:14<?, ?it/s]

(1578102, 257)
11
Mon Jun 10 10:10:19 2024


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |                                             | 0/? [00:18<?, ?it/s]

(1578102, 257)


In [ ]:
tracker.stop()

In [ ]:
time.sleep(30)

In [ ]:
#!sudo poweroff